In [1]:
import pandas as pd
import os

## Fill in header of the data sheet

In [2]:
fh = open("DataSheetHeader.csv")
datasheet_header = fh.read().rstrip()
fh.close()

In [3]:
datasheet_header = datasheet_header.replace("INVESTIGATOR", "Mathieu Claireaux")
datasheet_header = datasheet_header.replace("RUNNAME", "RunXXX-Mathieu")
datasheet_header = datasheet_header.replace("DATE", "20-01-2020")
print(datasheet_header)

[Header],,,,,,,,,,,,,,,
IEMFileVersion,4,,,,,,,,,,,,,,
Investigator Name,Mathieu Claireaux,,,,,,,,,,,,,,
Experiment Name,RunXXX-Mathieu,,,,,,,,,,,,,,
Date,20-01-2020,,,,,,,,,,,,,,
Workflow,GenerateFASTQ,,,,,,,,,,,,,,
Application,FASTQ Only,,,,,,,,,,,,,,
Assay,Nextera XT v2,,,,,,,,,,,,,,
Description,RNA,,,,,,,,,,,,,,
Chemistry,Amplicon,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
[Reads],,,,,,,,,,,,,,,
301,,,,,,,,,,,,,,,
301,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
[Settings],,,,,,,,,,,,,,,
ReverseComplement,0,,,,,,,,,,,,,,
Adapter,CTGTCTCTTATACACATCT,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
[Data],,,,,,,,,,,,,,,
Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Run,Nr,Sample_Project,Species,Chain,Patient,Sample,UMI


## Read file names from data directory on WebDav server

In [4]:
df_files = pd.read_csv("SAMPLES", sep="/", header=None)
df_files.columns = ["empty", "mnt", "immuno", "runs", "Run", "data", "Filename"]
df_files.head()

,empty,mnt,immuno,runs,Run,data,Filename
0,NaN,mnt,immunogenomics,RUNS,runXXX-20200121-mathieu,data,A_AlphaBC10_S0_L001.assembled-TAAGC-GAATGGTCAC...
1,NaN,mnt,immunogenomics,RUNS,runXXX-20200121-mathieu,data,A_AlphaBC11_S0_L001.assembled-TAAGC-CCAACGTCAC...
2,NaN,mnt,immunogenomics,RUNS,runXXX-20200121-mathieu,data,A_AlphaBC12_S0_L001.assembled-TAAGC-GAGACGTCAC...
3,NaN,mnt,immunogenomics,RUNS,runXXX-20200121-mathieu,data,A_AlphaBC1_S0_L001.assembled-TAAGC-GTTCAGTCAC....
4,NaN,mnt,immunogenomics,RUNS,runXXX-20200121-mathieu,data,A_AlphaBC2_S0_L001.assembled-TAAGC-CAGGAGTCAC....


In [5]:
cols = ["Run", "Filename"]
df_files = df_files[cols]
df_files.head()

,Run,Filename
0,runXXX-20200121-mathieu,A_AlphaBC10_S0_L001.assembled-TAAGC-GAATGGTCAC...
1,runXXX-20200121-mathieu,A_AlphaBC11_S0_L001.assembled-TAAGC-CCAACGTCAC...
2,runXXX-20200121-mathieu,A_AlphaBC12_S0_L001.assembled-TAAGC-GAGACGTCAC...
3,runXXX-20200121-mathieu,A_AlphaBC1_S0_L001.assembled-TAAGC-GTTCAGTCAC....
4,runXXX-20200121-mathieu,A_AlphaBC2_S0_L001.assembled-TAAGC-CAGGAGTCAC....


In [6]:
# Remove entries with "Undetermined" or "nomatch" in the file name
df_files = df_files[~df_files.Filename.str.contains("Undetermined")]
df_files = df_files[~df_files.Filename.str.contains("nomatch")]

In [7]:
f = lambda x: x.split("_L001")[0].split("_")[0]
df_files["abcd"] = [x for x in map(f, df_files["Filename"])]

In [8]:
f = lambda x: x.split("_L001")[0].split("_")[1]
df_files["alphabeta"] = [x for x in map(f, df_files["Filename"])]

In [9]:
f = lambda x: x.split("_L001")[0].split("_")[2]
df_files["Nr"] = [x for x in map(f, df_files["Filename"])]

In [10]:
df_files.head()

,Run,Filename,abcd,alphabeta,Nr
0,runXXX-20200121-mathieu,A_AlphaBC10_S0_L001.assembled-TAAGC-GAATGGTCAC...,A,AlphaBC10,S0
1,runXXX-20200121-mathieu,A_AlphaBC11_S0_L001.assembled-TAAGC-CCAACGTCAC...,A,AlphaBC11,S0
2,runXXX-20200121-mathieu,A_AlphaBC12_S0_L001.assembled-TAAGC-GAGACGTCAC...,A,AlphaBC12,S0
3,runXXX-20200121-mathieu,A_AlphaBC1_S0_L001.assembled-TAAGC-GTTCAGTCAC....,A,AlphaBC1,S0
4,runXXX-20200121-mathieu,A_AlphaBC2_S0_L001.assembled-TAAGC-CAGGAGTCAC....,A,AlphaBC2,S0


In [11]:
# Sample_ID,Sample_Name,Sample
df_files["Sample_Name"] = df_files["abcd"] + "_" + df_files["alphabeta"]
df_files["Sample_ID"] = df_files["Sample_Name"]
df_files["Sample"] = df_files["Sample_Name"]
df_files.head()

,Run,Filename,abcd,alphabeta,Nr,Sample_Name,Sample_ID,Sample
0,runXXX-20200121-mathieu,A_AlphaBC10_S0_L001.assembled-TAAGC-GAATGGTCAC...,A,AlphaBC10,S0,A_AlphaBC10,A_AlphaBC10,A_AlphaBC10
1,runXXX-20200121-mathieu,A_AlphaBC11_S0_L001.assembled-TAAGC-CCAACGTCAC...,A,AlphaBC11,S0,A_AlphaBC11,A_AlphaBC11,A_AlphaBC11
2,runXXX-20200121-mathieu,A_AlphaBC12_S0_L001.assembled-TAAGC-GAGACGTCAC...,A,AlphaBC12,S0,A_AlphaBC12,A_AlphaBC12,A_AlphaBC12
3,runXXX-20200121-mathieu,A_AlphaBC1_S0_L001.assembled-TAAGC-GTTCAGTCAC....,A,AlphaBC1,S0,A_AlphaBC1,A_AlphaBC1,A_AlphaBC1
4,runXXX-20200121-mathieu,A_AlphaBC2_S0_L001.assembled-TAAGC-CAGGAGTCAC....,A,AlphaBC2,S0,A_AlphaBC2,A_AlphaBC2,A_AlphaBC2


In [12]:
# Sample_Project
f = lambda x: "Mathieu-" + x.split("BC")[0]
df_files["Sample_Project"] = [x for x in map(f, df_files["alphabeta"])]
df_files.head()

,Run,Filename,abcd,alphabeta,Nr,Sample_Name,Sample_ID,Sample,Sample_Project
0,runXXX-20200121-mathieu,A_AlphaBC10_S0_L001.assembled-TAAGC-GAATGGTCAC...,A,AlphaBC10,S0,A_AlphaBC10,A_AlphaBC10,A_AlphaBC10,Mathieu-Alpha
1,runXXX-20200121-mathieu,A_AlphaBC11_S0_L001.assembled-TAAGC-CCAACGTCAC...,A,AlphaBC11,S0,A_AlphaBC11,A_AlphaBC11,A_AlphaBC11,Mathieu-Alpha
2,runXXX-20200121-mathieu,A_AlphaBC12_S0_L001.assembled-TAAGC-GAGACGTCAC...,A,AlphaBC12,S0,A_AlphaBC12,A_AlphaBC12,A_AlphaBC12,Mathieu-Alpha
3,runXXX-20200121-mathieu,A_AlphaBC1_S0_L001.assembled-TAAGC-GTTCAGTCAC....,A,AlphaBC1,S0,A_AlphaBC1,A_AlphaBC1,A_AlphaBC1,Mathieu-Alpha
4,runXXX-20200121-mathieu,A_AlphaBC2_S0_L001.assembled-TAAGC-CAGGAGTCAC....,A,AlphaBC2,S0,A_AlphaBC2,A_AlphaBC2,A_AlphaBC2,Mathieu-Alpha


In [13]:
# Species,Patient,UMI
df_files["Species"] = "human"
df_files["Patient"] = "Unknown"
df_files["UMI"] = "no"

In [14]:
# Chain
df_files["Chain"] = df_files["Sample_Project"]
df_files["Chain"] = df_files["Chain"].replace("Mathieu-Alpha", "TCRa")
df_files["Chain"] = df_files["Chain"].replace("Mathieu-Beta", "TCRb")
df_files

,Run,Filename,abcd,alphabeta,Nr,Sample_Name,Sample_ID,Sample,Sample_Project,Species,Patient,UMI,Chain
0,runXXX-20200121-mathieu,A_AlphaBC10_S0_L001.assembled-TAAGC-GAATGGTCAC...,A,AlphaBC10,S0,A_AlphaBC10,A_AlphaBC10,A_AlphaBC10,Mathieu-Alpha,human,Unknown,no,TCRa
1,runXXX-20200121-mathieu,A_AlphaBC11_S0_L001.assembled-TAAGC-CCAACGTCAC...,A,AlphaBC11,S0,A_AlphaBC11,A_AlphaBC11,A_AlphaBC11,Mathieu-Alpha,human,Unknown,no,TCRa
2,runXXX-20200121-mathieu,A_AlphaBC12_S0_L001.assembled-TAAGC-GAGACGTCAC...,A,AlphaBC12,S0,A_AlphaBC12,A_AlphaBC12,A_AlphaBC12,Mathieu-Alpha,human,Unknown,no,TCRa
3,runXXX-20200121-mathieu,A_AlphaBC1_S0_L001.assembled-TAAGC-GTTCAGTCAC....,A,AlphaBC1,S0,A_AlphaBC1,A_AlphaBC1,A_AlphaBC1,Mathieu-Alpha,human,Unknown,no,TCRa
4,runXXX-20200121-mathieu,A_AlphaBC2_S0_L001.assembled-TAAGC-CAGGAGTCAC....,A,AlphaBC2,S0,A_AlphaBC2,A_AlphaBC2,A_AlphaBC2,Mathieu-Alpha,human,Unknown,no,TCRa
5,runXXX-20200121-mathieu,A_AlphaBC3_S0_L001.assembled-TAAGC-TTATAGTCAC....,A,AlphaBC3,S0,A_AlphaBC3,A_AlphaBC3,A_AlphaBC3,Mathieu-Alpha,human,Unknown,no,TCRa
6,runXXX-20200121-mathieu,A_AlphaBC4_S0_L001.assembled-TAAGC-CCTGTGTCAC....,A,AlphaBC4,S0,A_AlphaBC4,A_AlphaBC4,A_AlphaBC4,Mathieu-Alpha,human,Unknown,no,TCRa
7,runXXX-20200121-mathieu,A_AlphaBC5_S0_L001.assembled-TAAGC-ACCGCGTCAC....,A,AlphaBC5,S0,A_AlphaBC5,A_AlphaBC5,A_AlphaBC5,Mathieu-Alpha,human,Unknown,no,TCRa
8,runXXX-20200121-mathieu,A_AlphaBC6_S0_L001.assembled-TAAGC-ACTTAGTCAC....,A,AlphaBC6,S0,A_AlphaBC6,A_AlphaBC6,A_AlphaBC6,Mathieu-Alpha,human,Unknown,no,TCRa
9,runXXX-20200121-mathieu,A_AlphaBC7_S0_L001.assembled-TAAGC-GCTAGGTCAC....,A,AlphaBC7,S0,A_AlphaBC7,A_AlphaBC7,A_AlphaBC7,Mathieu-Alpha,human,Unknown,no,TCRa


In [15]:
# Empty columns for: Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2
df_files["Sample_Plate"] = ""
df_files["Sample_Well"] = ""
df_files["I7_Index_ID"] = ""
df_files["index"] = ""
df_files["I5_Index_ID"] = ""
df_files["index2"] = ""

In [16]:
# Change order of the columns
cols = ["Sample_ID","Sample_Name","Sample_Plate","Sample_Well","I7_Index_ID","index","I5_Index_ID","index2","Run","Nr","Sample_Project","Species","Chain","Patient","Sample","UMI","Filename"]
df_files = df_files[cols]
df_files

,Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Run,Nr,Sample_Project,Species,Chain,Patient,Sample,UMI,Filename
0,A_AlphaBC10,A_AlphaBC10,,,,,,,runXXX-20200121-mathieu,S0,Mathieu-Alpha,human,TCRa,Unknown,A_AlphaBC10,no,A_AlphaBC10_S0_L001.assembled-TAAGC-GAATGGTCAC...
1,A_AlphaBC11,A_AlphaBC11,,,,,,,runXXX-20200121-mathieu,S0,Mathieu-Alpha,human,TCRa,Unknown,A_AlphaBC11,no,A_AlphaBC11_S0_L001.assembled-TAAGC-CCAACGTCAC...
2,A_AlphaBC12,A_AlphaBC12,,,,,,,runXXX-20200121-mathieu,S0,Mathieu-Alpha,human,TCRa,Unknown,A_AlphaBC12,no,A_AlphaBC12_S0_L001.assembled-TAAGC-GAGACGTCAC...
3,A_AlphaBC1,A_AlphaBC1,,,,,,,runXXX-20200121-mathieu,S0,Mathieu-Alpha,human,TCRa,Unknown,A_AlphaBC1,no,A_AlphaBC1_S0_L001.assembled-TAAGC-GTTCAGTCAC....
4,A_AlphaBC2,A_AlphaBC2,,,,,,,runXXX-20200121-mathieu,S0,Mathieu-Alpha,human,TCRa,Unknown,A_AlphaBC2,no,A_AlphaBC2_S0_L001.assembled-TAAGC-CAGGAGTCAC....
5,A_AlphaBC3,A_AlphaBC3,,,,,,,runXXX-20200121-mathieu,S0,Mathieu-Alpha,human,TCRa,Unknown,A_AlphaBC3,no,A_AlphaBC3_S0_L001.assembled-TAAGC-TTATAGTCAC....
6,A_AlphaBC4,A_AlphaBC4,,,,,,,runXXX-20200121-mathieu,S0,Mathieu-Alpha,human,TCRa,Unknown,A_AlphaBC4,no,A_AlphaBC4_S0_L001.assembled-TAAGC-CCTGTGTCAC....
7,A_AlphaBC5,A_AlphaBC5,,,,,,,runXXX-20200121-mathieu,S0,Mathieu-Alpha,human,TCRa,Unknown,A_AlphaBC5,no,A_AlphaBC5_S0_L001.assembled-TAAGC-ACCGCGTCAC....
8,A_AlphaBC6,A_AlphaBC6,,,,,,,runXXX-20200121-mathieu,S0,Mathieu-Alpha,human,TCRa,Unknown,A_AlphaBC6,no,A_AlphaBC6_S0_L001.assembled-TAAGC-ACTTAGTCAC....
9,A_AlphaBC7,A_AlphaBC7,,,,,,,runXXX-20200121-mathieu,S0,Mathieu-Alpha,human,TCRa,Unknown,A_AlphaBC7,no,A_AlphaBC7_S0_L001.assembled-TAAGC-GCTAGGTCAC....


# Glue together the header and the values

In [17]:
df_files.to_csv("tmp.csv", header=None, index=None)

In [18]:
fh = open("tmp.csv")
values = fh.read().rstrip()
fh.close()

In [19]:
fhOut = open("20200120-DataSheet-Mathieu.csv", "w")
print(datasheet_header + ",filename", file=fhOut)
print(values, file=fhOut)
fhOut.close()
print("Wrote 20200120-DataSheet-Mathieu.csv to disk")

Wrote 20200120-DataSheet-Mathieu.csv to disk
